In [1]:
from pyspark import SparkContext
import pandas as pd

sc = SparkContext("local", "diabetes")

21/08/25 22:42:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
data = pd.read_csv('diabetes.csv')
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [4]:
sparkDF = sqlContext.createDataFrame(data)
sparkDF.printSchema()
sparkDF.show()

root
 |-- Pregnancies: long (nullable = true)
 |-- Glucose: long (nullable = true)
 |-- BloodPressure: long (nullable = true)
 |-- SkinThickness: long (nullable = true)
 |-- Insulin: long (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: long (nullable = true)
 |-- Outcome: long (nullable = true)



+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|31.0|                   0.248| 26|      1|


In [5]:
sparkDF = sparkDF.filter(sparkDF['BloodPressure'] > 0)
sparkDF = sparkDF.filter(sparkDF['SkinThickness'] > 0)
sparkDF = sparkDF.filter(sparkDF['Insulin'] > 0)
sparkDF.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          3|     78|           50|           32|     88|31.0|                   0.248| 26|      1|
|          2|    197|           70|           45|    543|30.5|                   0.158| 53|      1|
|          1|    189|           60|           23|    846|30.1|                   0.398| 59|      1|
|          5|    166|           72|           19|    175|25.8|                   0.587| 51|      1|
|          0|    118|           84|           47|    230|45.8|                   0.551| 31|      1|


In [6]:
sparkDF.count()

394

In [10]:
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler(inputCols=['Pregnancies', 'Glucose', 'BloodPressure', \
                                'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'] \
                     , outputCol='features')
df = va.transform(sparkDF)
df = df.select(['features', 'Outcome'])
df.show()

+--------------------+-------+
|            features|Outcome|
+--------------------+-------+
|[1.0,89.0,66.0,23...|      0|
|[0.0,137.0,40.0,3...|      1|
|[3.0,78.0,50.0,32...|      1|
|[2.0,197.0,70.0,4...|      1|
|[1.0,189.0,60.0,2...|      1|
|[5.0,166.0,72.0,1...|      1|
|[0.0,118.0,84.0,4...|      1|
|[1.0,103.0,30.0,3...|      0|
|[1.0,115.0,70.0,3...|      1|
|[3.0,126.0,88.0,4...|      0|
|[11.0,143.0,94.0,...|      1|
|[10.0,125.0,70.0,...|      1|
|[1.0,97.0,66.0,15...|      0|
|[13.0,145.0,82.0,...|      0|
|[3.0,158.0,76.0,3...|      1|
|[3.0,88.0,58.0,11...|      0|
|[4.0,103.0,60.0,3...|      0|
|[4.0,111.0,72.0,4...|      1|
|[3.0,180.0,64.0,2...|      0|
|[9.0,171.0,110.0,...|      1|
+--------------------+-------+
only showing top 20 rows



In [13]:
split = df.randomSplit([0.7, 0.3])
trainDF = split[0]
testDF = split[1]

In [14]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = 'features', labelCol='Outcome')

model = rf.fit(trainDF)

In [19]:
predictions = model.transform(testDF)
selected  = predictions.select(['Outcome', 'prediction'])
selected.show()

+-------+----------+
|Outcome|prediction|
+-------+----------+
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       1.0|
|      0|       0.0|
|      0|       0.0|
|      1|       0.0|
|      0|       0.0|
|      1|       0.0|
|      0|       0.0|
|      0|       0.0|
|      1|       1.0|
|      0|       0.0|
|      0|       0.0|
|      0|       1.0|
|      1|       0.0|
|      0|       1.0|
|      0|       1.0|
|      1|       1.0|
+-------+----------+
only showing top 20 rows



In [20]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol='Outcome', \
                                              predictionCol='prediction',\
                                              metricName="accuracy")
accuracy = evaluator.evaluate(selected)
accuracy

0.7734375

In [26]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans().setK(5)

In [27]:
model = kmeans.fit(trainDF)

predictions = model.transform(testDF)

In [28]:
predictions.show()

+--------------------+-------+----------+
|            features|Outcome|prediction|
+--------------------+-------+----------+
|[0.0,84.0,64.0,22...|      0|         4|
|[0.0,93.0,60.0,25...|      0|         4|
|[0.0,93.0,100.0,3...|      0|         4|
|[0.0,95.0,80.0,45...|      0|         4|
|[0.0,100.0,88.0,6...|      0|         1|
|[0.0,102.0,86.0,1...|      0|         1|
|[0.0,104.0,64.0,2...|      0|         1|
|[0.0,104.0,64.0,3...|      1|         4|
|[0.0,106.0,70.0,3...|      0|         1|
|[0.0,121.0,66.0,3...|      1|         2|
|[0.0,126.0,84.0,2...|      0|         2|
|[0.0,127.0,80.0,3...|      0|         2|
|[0.0,129.0,110.0,...|      1|         1|
|[0.0,134.0,58.0,2...|      0|         0|
|[0.0,139.0,62.0,1...|      0|         2|
|[0.0,152.0,82.0,3...|      0|         0|
|[0.0,162.0,76.0,5...|      1|         1|
|[0.0,165.0,76.0,4...|      0|         0|
|[0.0,165.0,90.0,3...|      0|         3|
|[0.0,177.0,60.0,2...|      1|         3|
+--------------------+-------+----

In [30]:
centers = model.clusterCenters()

for center in centers:
    print(center)

[  3.91666667 152.95833333  72.16666667  29.08333333 311.875
  35.05833333   0.63533333  34.04166667]
[  3.47560976 125.35365854  71.65853659  28.09756098 121.75609756
  32.72560976   0.46946341  31.54878049]
[  3.34920635 133.3015873   71.96825397  31.3015873  197.17460317
  35.14920635   0.61349206  32.76190476]
[  4.36363636 163.54545455  75.63636364  34.54545455 533.45454545
  34.23636364   0.57854545  40.63636364]
[ 2.39534884 95.93023256 67.96511628 26.15116279 63.41860465 30.69418605
  0.47330233 27.02325581]
